Import Dependencies

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Custom packages
import lib.global_settings as settings
from doc_customizer import doc_customization
from lib.helper_funcs import read_markdown_file
from components.BinaryClassifier.main_binaryclassifier import question_classifier

# Other dependencies
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from langsmith import Client

In [2]:
# Import the Stack Overflow with accepted answers dataset
df = pd.read_csv(os.path.join(settings.DATA_PATH, "DocQues_AcceptedAns_Issues_v2_2018_2023.csv"))
df = df[["QuestionId", "QuestionAPI", "IssueType", "Title", "Body", "AcceptedAnswer"]]
df = df.rename(columns={"Body":"Question", "AcceptedAnswer":"GroundTruth"})

indexes = []
for index, row in tqdm(df.iterrows()):
    if row['QuestionAPI'].startswith("tf."):
        documentation = read_markdown_file(row['QuestionAPI'])
        if documentation is not None:
            pred = question_classifier(row['Question'])
            if pred == 1:
                indexes.append(index)

# Pick first record only for testing
df = df.loc[indexes]
df = df.reset_index()
del df['index']
df

126it [03:07,  1.49s/it]


,QuestionId,QuestionAPI,IssueType,Title,Question,GroundTruth
0,76324368,tf.keras.layers.Dense,Documentation Replication on Other Examples,Understanding tf.keras.layers.Dense(),<p>I am trying to understand why there is a di...,<p>After some experimentation I realized that ...
1,74005009,tf.data.Dataset,Documentation Replicability,How to create output_signature for tensorflow....,<p>I have a generator yielding data and labels...,<p>if your datagen_row() function yields input...
2,71335830,tf.keras.layers.Flatten,Documentation Ambiguity,What is the difference between tf.keras.layers...,<p>I have seen multiple uses of both <code>tf....,<p>I think the confusion comes from using a <c...
3,71149271,tf.data.Dataset,Documentation Replication on Other Examples,How to remove single feature from tensorflow d...,<p>I created dataset from csv file with datase...,<p>You can remove features by only filtering t...
4,71129505,tf.data.Dataset,Documentation Replication on Other Examples,Is it possible to split a tensorflow dataset i...,<p>I am using <code>tf.keras.utils.image_datas...,<p>The issue is that you are not taking and sk...
5,71019644,tf.where,Documentation Ambiguity,Equivalent tensorflow expression to numpy mask,<p>I have a numpy array named PixelData of unk...,<p>Not sure what <code>PixelData</code> looks ...
6,70880589,tf.data.experimental.cardinality,Lack of Alternative Solutions/Documentation,what does cardinality mean in relation to an i...,<p>After successfully creating a tensorflow im...,"<p>The cardinality, in your case, is simply th..."
7,70747499,tf.map_fn,Documentation Replication on Other Examples,Using tf.map_fn when the function has multiple...,<p>I can easily use tf.map_fn when the functio...,<p>You should make sure you are returning a te...
8,68984841,tf.keras.layers.Dense,Documentation Replicability,How can I understand the kernel of tf.keras.la...,<p>How can I understand the kernel of <code>tf...,"<p>Yes, your understanding is correct.</p>\n<p..."
9,68431633,tf.image.stateless_random_crop,Documentation Ambiguity,tf.image.stateless_random_crop VS. tf.image.ra...,"<p>In tf 2.5, there are two functions for crop...",<p><code>random_crop</code> always return the ...


In [3]:
df.to_csv(os.path.join(settings.DATA_PATH, "eval_data_ragas.csv"), index=False)

In [4]:
df['IssueType'].value_counts()

IssueType
Documentation Replication on Other Examples    24
Documentation Replicability                    11
Documentation Ambiguity                        10
Lack of Alternative Solutions/Documentation     6
Documentation Completeness                      2
Inadequate Examples                             1
Name: count, dtype: int64

In [2]:
df = pd.read_csv(os.path.join(settings.DATA_PATH, "eval_data_ragas.csv"))
new_cols = {'Answer': str, 'Context': list, 'Intent': str,}
for col_name, dtype in new_cols.items():
  df[col_name] = np.empty(len(df), dtype=dtype)
column_name = 'IssueType'
category1 = "Documentation Replication on Other Examples"
category2 = "Documentation Replicability"
filtered_df = df[df[column_name].isin([category1,category2 ])]

# Sample rows from each category
category1_sample = filtered_df[filtered_df[column_name] == category1].sample(5)
category2_sample = filtered_df[filtered_df[column_name] == category2].sample(5)

# Combine samples and return
eval_data =  pd.concat([category1_sample, category2_sample])
eval_data = eval_data.reset_index()
del eval_data['index']
eval_data


,QuestionId,QuestionAPI,IssueType,Title,Question,GroundTruth,Answer,Context,Intent
0,55909188,tf.nn.conv2d,Documentation Replication on Other Examples,How can I apply a TensorFlow 2D Convolution (t...,<p>I would like to use the function <code>tf.n...,<p>AFAIK there is no way around it. It seems (...,,None,
1,53032922,tf.while_loop,Documentation Replication on Other Examples,TensorFlow while loop with condition dependent...,<p>I want to have a while loop with the condit...,<p>The arguments that are passed on to the <co...,,None,
2,59555206,tf.keras,Documentation Replication on Other Examples,keras to tf.keras Conversion: Dense layer dime...,<p>So I've built a convnet using pure <code>ke...,<p>There's few issues with your code. Fix them...,,None,
3,48914952,tf.unique,Documentation Replication on Other Examples,num_buckets as a parameter in a tensorflow fea...,<p>Currently Tensorflow documentation define a...,<p>If <code>list_of_unique_values_in_the_colum...,,None,
4,59361689,tf.keras.backend,Documentation Replication on Other Examples,Redundancies in tf.keras.backend and tensorflo...,<p>I have been working in TensorFlow for about...,<p>Short answer: Prefer tensorflow's native AP...,,None,
5,74005009,tf.data.Dataset,Documentation Replicability,How to create output_signature for tensorflow....,<p>I have a generator yielding data and labels...,<p>if your datagen_row() function yields input...,,None,
6,57449484,tf.compat.v1.layers.batch_normalization,Documentation Replicability,What is trainable parameter in tensorflow?,<p>tf.compat.v1.layers.batch_normalization tak...,"<p>First of all, this function is <a href=""htt...",,None,
7,63004540,tf.pad,Documentation Replicability,How to pad 1 dimensinal vector in tensorflow? ...,<p>I am trying to use tf.pad. Here is my attem...,<p>You have to specify the padding at the begi...,,None,
8,68984841,tf.keras.layers.Dense,Documentation Replicability,How can I understand the kernel of tf.keras.la...,<p>How can I understand the kernel of <code>tf...,"<p>Yes, your understanding is correct.</p>\n<p...",,None,
9,55560676,tf.while_loop,Documentation Replicability,How to use tf.while_loop with eager execution?,"<p>In the documentation, the body of a tf.whil...",<p>You forgot to add return statement to your ...,,None,


In [3]:
df = eval_data.iloc[[0]]
df

,QuestionId,QuestionAPI,IssueType,Title,Question,GroundTruth,Answer,Context,Intent
0,55909188,tf.nn.conv2d,Documentation Replication on Other Examples,How can I apply a TensorFlow 2D Convolution (t...,<p>I would like to use the function <code>tf.n...,<p>AFAIK there is no way around it. It seems (...,,None,


# Retrieval Augmented Generation Assesment (RAGAS)

In [20]:
for i in range(len(eval_data)):
    try:
        print(f"{'='*50} Test Record: {i} {'='*50}")
        eval_data = doc_customization(eval_data, i)
        print(f"{'='*120}\n")
    except:
        pass

================================================== Test Record: 0 ==================================================
🚀: EXECUTING INTENT IDENTIFIER: Identifying intent of the SO question


/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


✅: EXECUTION COMPLETED

🚀: INITIATING WEB SEARCH: Tavily Search API + GPT-4-Turbo


/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


✅: EXECUTION COMPLETED

🚀: EXECUTING RETRIEVER: Cohere RAG Retriever
	➡️ Relevant Parent Doc Count: 0
✅: EXECUTION COMPLETED

🚀: EXECUTING Q&A RETRIEVER: Searching for relevant SO Q&As with accepted answers.
	➡️ Relevant Stack Overflow Q&A Count: 1


...

/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


✅: CUSTOMIZATION COMPLETED


================================================== Test Record: 9 ==================================================
🚀: EXECUTING INTENT IDENTIFIER: Identifying intent of the SO question


/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


✅: EXECUTION COMPLETED

🚀: INITIATING WEB SEARCH: Tavily Search API + GPT-4-Turbo


/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [21]:
eval_data.to_csv(os.path.join(settings.DATA_PATH, "eval_data_ragas_with_results.csv"), index=False)
eval_data

,QuestionId,QuestionAPI,IssueType,Title,Question,GroundTruth,Answer,Context,Intent
0,55909188,tf.nn.conv2d,Documentation Replication on Other Examples,How can I apply a TensorFlow 2D Convolution (t...,<p>I would like to use the function <code>tf.n...,<p>AFAIK there is no way around it. It seems (...,# Customized Content\n\nTo apply the `tf.nn.co...,[],The user is seeking a simpler or standard meth...
1,53032922,tf.while_loop,Documentation Replication on Other Examples,TensorFlow while loop with condition dependent...,<p>I want to have a while loop with the condit...,<p>The arguments that are passed on to the <co...,# Customized Content\n\nTo address your questi...,[Stack Overflow Public questions & answers\n\n...,The user is trying to understand how to implem...
2,59555206,tf.keras,Documentation Replication on Other Examples,keras to tf.keras Conversion: Dense layer dime...,<p>So I've built a convnet using pure <code>ke...,<p>There's few issues with your code. Fix them...,# Customized Content\n\nThe error you're encou...,[None found for the last dimension in the inpu...,The user is trying to understand why they are ...
3,48914952,tf.unique,Documentation Replication on Other Examples,num_buckets as a parameter in a tensorflow fea...,<p>Currently Tensorflow documentation define a...,<p>If <code>list_of_unique_values_in_the_colum...,# Customized Content\n\nTo automate the proces...,[Specifying a three-element boundaries vector ...,The user is seeking a method to automatically ...
4,59361689,tf.keras.backend,Documentation Replication on Other Examples,Redundancies in tf.keras.backend and tensorflo...,<p>I have been working in TensorFlow for about...,<p>Short answer: Prefer tensorflow's native AP...,# Customized Content\n\nUnderstanding when to ...,"[– star Jan 3, 2020 at 5:18\n\nThe first point...",The user is trying to understand when to use t...
5,74005009,tf.data.Dataset,Documentation Replicability,How to create output_signature for tensorflow....,<p>I have a generator yielding data and labels...,<p>if your datagen_row() function yields input...,"Based on your description, it seems you are co...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to understand how to corre...
6,57449484,tf.compat.v1.layers.batch_normalization,Documentation Replicability,What is trainable parameter in tensorflow?,<p>tf.compat.v1.layers.batch_normalization tak...,"<p>First of all, this function is <a href=""htt...","# Customized Content\n\nIn TensorFlow, a train...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to understand how the 'tra...
7,63004540,tf.pad,Documentation Replicability,How to pad 1 dimensinal vector in tensorflow? ...,<p>I am trying to use tf.pad. Here is my attem...,<p>You have to specify the padding at the begi...,# Customized Content\n\nTo pad a 1-dimensional...,[Stack Overflow Public questions & answers\n\n...,The user is trying to understand how to correc...
8,68984841,tf.keras.layers.Dense,Documentation Replicability,How can I understand the kernel of tf.keras.la...,<p>How can I understand the kernel of <code>tf...,"<p>Yes, your understanding is correct.</p>\n<p...","# Customized Content\n\nYes, your understandin...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to confirm their understan...
9,55560676,tf.while_loop,Documentation Replicability,How to use tf.while_loop with eager execution?,"<p>In the documentation, the body of a tf.whil...",<p>You forgot to add return statement to your ...,,None,The user is trying to understand why their cod...


In [24]:
indices_to_drop = [0,9]
df = eval_data.drop(indices_to_drop)
df

,QuestionId,QuestionAPI,IssueType,Title,Question,GroundTruth,Answer,Context,Intent
1,53032922,tf.while_loop,Documentation Replication on Other Examples,TensorFlow while loop with condition dependent...,<p>I want to have a while loop with the condit...,<p>The arguments that are passed on to the <co...,# Customized Content\n\nTo address your questi...,[Stack Overflow Public questions & answers\n\n...,The user is trying to understand how to implem...
2,59555206,tf.keras,Documentation Replication on Other Examples,keras to tf.keras Conversion: Dense layer dime...,<p>So I've built a convnet using pure <code>ke...,<p>There's few issues with your code. Fix them...,# Customized Content\n\nThe error you're encou...,[None found for the last dimension in the inpu...,The user is trying to understand why they are ...
3,48914952,tf.unique,Documentation Replication on Other Examples,num_buckets as a parameter in a tensorflow fea...,<p>Currently Tensorflow documentation define a...,<p>If <code>list_of_unique_values_in_the_colum...,# Customized Content\n\nTo automate the proces...,[Specifying a three-element boundaries vector ...,The user is seeking a method to automatically ...
4,59361689,tf.keras.backend,Documentation Replication on Other Examples,Redundancies in tf.keras.backend and tensorflo...,<p>I have been working in TensorFlow for about...,<p>Short answer: Prefer tensorflow's native AP...,# Customized Content\n\nUnderstanding when to ...,"[– star Jan 3, 2020 at 5:18\n\nThe first point...",The user is trying to understand when to use t...
5,74005009,tf.data.Dataset,Documentation Replicability,How to create output_signature for tensorflow....,<p>I have a generator yielding data and labels...,<p>if your datagen_row() function yields input...,"Based on your description, it seems you are co...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to understand how to corre...
6,57449484,tf.compat.v1.layers.batch_normalization,Documentation Replicability,What is trainable parameter in tensorflow?,<p>tf.compat.v1.layers.batch_normalization tak...,"<p>First of all, this function is <a href=""htt...","# Customized Content\n\nIn TensorFlow, a train...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to understand how the 'tra...
7,63004540,tf.pad,Documentation Replicability,How to pad 1 dimensinal vector in tensorflow? ...,<p>I am trying to use tf.pad. Here is my attem...,<p>You have to specify the padding at the begi...,# Customized Content\n\nTo pad a 1-dimensional...,[Stack Overflow Public questions & answers\n\n...,The user is trying to understand how to correc...
8,68984841,tf.keras.layers.Dense,Documentation Replicability,How can I understand the kernel of tf.keras.la...,<p>How can I understand the kernel of <code>tf...,"<p>Yes, your understanding is correct.</p>\n<p...","# Customized Content\n\nYes, your understandin...",[Stack Overflow Public questions & answers\n\n...,The user is seeking to confirm their understan...


In [25]:
from datasets import Dataset

# Wrap dataset in a hugging face dataset
response_dataset = Dataset.from_dict({
    "question" : df['Question'],
    "answer" : df['Answer'],
    "contexts" : df['Context'],
    "ground_truth" : df['GroundTruth']
})

In [26]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

metrics = [
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
]

In [27]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sharukat/Documents/ResearchYU/Code/doc-customizer-llm/.venv/lib/python3.11/site-packages/pydantic

In [28]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision,answer_correctness,answer_similarity
0,<p>I want to have a while loop with the condit...,# Customized Content\n\nTo address your questi...,[Stack Overflow Public questions & answers\n\n...,<p>The arguments that are passed on to the <co...,0.836663,1.0,1.0,1.0,0.433573,0.916109
1,<p>So I've built a convnet using pure <code>ke...,# Customized Content\n\nThe error you're encou...,[None found for the last dimension in the inpu...,<p>There's few issues with your code. Fix them...,0.741113,1.0,1.0,1.0,0.620286,0.902303
2,<p>Currently Tensorflow documentation define a...,# Customized Content\n\nTo automate the proces...,[Specifying a three-element boundaries vector ...,<p>If <code>list_of_unique_values_in_the_colum...,0.822665,1.0,1.0,1.0,0.810174,0.840696
3,<p>I have been working in TensorFlow for about...,# Customized Content\n\nUnderstanding when to ...,"[– star Jan 3, 2020 at 5:18\n\nThe first point...",<p>Short answer: Prefer tensorflow's native AP...,0.815139,1.0,1.0,1.0,0.858384,0.907233
4,<p>I have a generator yielding data and labels...,"Based on your description, it seems you are co...",[Stack Overflow Public questions & answers\n\n...,<p>if your datagen_row() function yields input...,0.815652,1.0,1.0,1.0,0.814646,0.858585
5,<p>tf.compat.v1.layers.batch_normalization tak...,"# Customized Content\n\nIn TensorFlow, a train...",[Stack Overflow Public questions & answers\n\n...,"<p>First of all, this function is <a href=""htt...",0.872250,1.0,1.0,1.0,0.598906,0.895624
6,<p>I am trying to use tf.pad. Here is my attem...,# Customized Content\n\nTo pad a 1-dimensional...,[Stack Overflow Public questions & answers\n\n...,<p>You have to specify the padding at the begi...,0.812715,1.0,1.0,1.0,0.860409,0.870207
7,<p>How can I understand the kernel of <code>tf...,"# Customized Content\n\nYes, your understandin...",[Stack Overflow Public questions & answers\n\n...,"<p>Yes, your understanding is correct.</p>\n<p...",0.828190,1.0,1.0,1.0,0.338759,0.886206


In [30]:
client = Client()
dataset_name = "RAGAS Evaluation Dataset"

dataset = client.upload_dataframe(
    df=results_df,
    input_keys=["question", "answer", "contexts", "ground_truth"],
    output_keys=["answer_relevancy", "faithfulness", "context_recall", "context_precision", "answer_correctness", "answer_similarity"],
    name=dataset_name,
    description="Evaluation results of the RAG application.",
    data_type="kv" # The default
)